In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print("imported libraries")

imported libraries


Reading the dataset

In [29]:
data = pd.read_csv("train.csv")
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


Pre-processing of data: Removing columns which have no impact on deciding the survival of passengers.  

In [30]:
data.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
data.head(3)

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250


Dividing the dataset into input and target

In [31]:
target=data.Survived
inputs=data.drop('Survived', axis='columns')

Changing the values of sex columns into numbers so that we can use them to train our model.

In [32]:
dummies=pd.get_dummies(inputs.Sex)

Merging the male female column into the original dataset.

In [33]:
inputs=pd.concat([inputs,dummies],axis='columns')

Deleting Sex column as no longer required.

In [34]:
inputs.drop('Sex', axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0


Checking if any column has NA values

In [35]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

Fill the NA value by taking avg of all Age's numeric value.

In [36]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())

Spliting the dataset into training and testing dataset

In [37]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test , y_train = train_test_split(inputs,target,test_size=0.2)

Calculate P(X=x/Y=y) using Gaussian distribution

In [38]:
def calculate_likelihood_gaussian(X, Y, label, feat_name, feat_val):
    x = X[feat_name]
    count = 0
    for index in x.index:
        if Y[index]==label and x[index]==feat_val:
            count=count+1
    p_x_and_y = count/len(x)
    p_y = len(Y[Y==label])/len(Y)
    return p_x_and_y/p_y

Calculate P(X=x1/Y=y)P(X=x2/Y=y).....P(X=xn/Y=y)*P()

In [39]:
def pxgiveny(X, Y, index, label):
    count=1
    columns = X.columns
    for i in columns:
        count=count*calculate_likelihood_gaussian(X, Y, label, i, X.loc[index][i])
    return count*len(Y[Y==label])/len(Y)

Function to find accuracy of prediction on test data set

In [40]:
def p(X, Y):
    a=[]
    for ind in X.index:
        c = sorted(list(Y.unique()))
        max = 0
        for i in c:
          p =  pxgiveny(X, Y, ind, i)
          if p>max :
            max = i
        a.append(max)
    Pr_df = pd.DataFrame({'yPredict': a, 'y': Y})
    count = 0
    yPre = Pr_df.yPredict
    y = Pr_df.y
    for i in Pr_df.index:
        if  y[i]==yPre[i]:
            count = count+1
    accuracy = (count/len(Y))*100
    print('accuracy:', accuracy)
    return accuracy

In [41]:
p(X_test, y_test)

accuracy: 66.43258426966293


66.43258426966293